# Hands-On Section (HW2)
## Fitting models and overfitting  + using scripts




### Downloading & Installing Packages
**READ THIS OR YOU WILL GET ERRORS**

There are some packages that will be required for this assignment that are no pre-loaded into Jupyter hub. You can install them following the instructions below.

Enter the following commands into jupyter's terminal one at a time:
```
sudo pip install liac-arff
sudo apt install graphviz
sudo pip install graphviz
sudo pip install pydotplus
```
enter "`Y`" when promted.

# Hands-On Section (HW2)
Import the churn training and test data in the data folder with pandas' read_csv function


In [2]:
! sudo pip install liac-arff

The directory '/home/ubuntu/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/ubuntu/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


In [16]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import datasets
import arff

In [17]:
arff_train = arff.load(open('data/churn2_train.arff'))
train = pd.DataFrame(arff_train['data'])
train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,one,119070.0,25.0,17.0,179158.0,541.0,4.0,4.0,very_unsat,very_little,actively_looking_into_it,3.42,LEAVE
1,one,40157.0,53.0,0.0,972459.0,220.0,26.0,8.0,unsat,high,considering,5.35,STAY
2,zero,116959.0,211.0,0.0,245177.0,895.0,15.0,15.0,very_sat,very_little,actively_looking_into_it,3.51,LEAVE
3,one,21118.0,52.0,7.0,413655.0,205.0,3.0,5.0,very_sat,little,no,4.21,LEAVE
4,one,58114.0,154.0,50.0,258444.0,334.0,26.0,1.0,unsat,high,considering,3.14,LEAVE


In [18]:
# let's give the columns their proper names
a = arff_train['attributes']
train.columns = [i[0] for i in a] 
train.head()





,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,PROFITABILITY,LEAVE
0,one,119070.0,25.0,17.0,179158.0,541.0,4.0,4.0,very_unsat,very_little,actively_looking_into_it,3.42,LEAVE
1,one,40157.0,53.0,0.0,972459.0,220.0,26.0,8.0,unsat,high,considering,5.35,STAY
2,zero,116959.0,211.0,0.0,245177.0,895.0,15.0,15.0,very_sat,very_little,actively_looking_into_it,3.51,LEAVE
3,one,21118.0,52.0,7.0,413655.0,205.0,3.0,5.0,very_sat,little,no,4.21,LEAVE
4,one,58114.0,154.0,50.0,258444.0,334.0,26.0,1.0,unsat,high,considering,3.14,LEAVE


### Fitting the Decision Tree
In Weka we used the J48 algorithm that implements the C4.5 algorithm. Sklearn instead uses the CART algorithm which does not work with categorical or nominal data. Let us convert each of these categorical variables to numeric variables.

In [19]:
#look at the values present in each of the categorical columns (float)
print(train['COLLEGE'].unique())
print(train['REPORTED_SATISFACTION'].unique())
print(train['REPORTED_USAGE_LEVEL'].unique())
print(train['CONSIDERING_CHANGE_OF_PLAN'].unique())
print(train['LEAVE'].unique())


['one' 'zero']
['very_unsat' 'unsat' 'very_sat' 'sat' 'avg']
['very_little' 'high' 'little' 'avg' 'very_high']
['actively_looking_into_it' 'considering' 'no' 'perhaps' 'never_thought']
['LEAVE' 'STAY']


In [20]:
train.head()

,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,PROFITABILITY,LEAVE
0,one,119070.0,25.0,17.0,179158.0,541.0,4.0,4.0,very_unsat,very_little,actively_looking_into_it,3.42,LEAVE
1,one,40157.0,53.0,0.0,972459.0,220.0,26.0,8.0,unsat,high,considering,5.35,STAY
2,zero,116959.0,211.0,0.0,245177.0,895.0,15.0,15.0,very_sat,very_little,actively_looking_into_it,3.51,LEAVE
3,one,21118.0,52.0,7.0,413655.0,205.0,3.0,5.0,very_sat,little,no,4.21,LEAVE
4,one,58114.0,154.0,50.0,258444.0,334.0,26.0,1.0,unsat,high,considering,3.14,LEAVE


In [21]:
# give these all numeric labels now: 

# create a dictionary mapping each string to a value
to_replace = {'COLLEGE':{'one':1,'zero':0},
           'REPORTED_SATISFACTION':{'very_unsat':-2,'unsat':-1,'avg':0,'sat':1,'very_sat':2},
           'REPORTED_USAGE_LEVEL':{'very_little':-2,'little':-1,'avg':0,'high':1,'very_high':2},
           'CONSIDERING_CHANGE_OF_PLAN':{'no':-2,'never_thought':-1,'perhaps':0,'considering':1,'actively_looking_into_it':2},
           'LEAVE':{'LEAVE':1,'STAY':0}
          }

train = train.replace(to_replace)

# note that if you run this cell twice it will crash. If this crashes for you reload the churn data.

In [22]:
train.head()

,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,PROFITABILITY,LEAVE
0,1,119070.0,25.0,17.0,179158.0,541.0,4.0,4.0,-2,-2,2,3.42,1
1,1,40157.0,53.0,0.0,972459.0,220.0,26.0,8.0,-1,1,1,5.35,0
2,0,116959.0,211.0,0.0,245177.0,895.0,15.0,15.0,2,-2,2,3.51,1
3,1,21118.0,52.0,7.0,413655.0,205.0,3.0,5.0,2,-1,-2,4.21,1
4,1,58114.0,154.0,50.0,258444.0,334.0,26.0,1.0,-1,1,1,3.14,1


In [23]:
Y_train = train["LEAVE"]
X_train = train.loc[:, train.columns != "LEAVE"]

my_tree = DecisionTreeClassifier()
my_tree.fit(X_train,Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

Now let's import the test data and transform it in the same way

In [24]:
arff_test = arff.load(open('data/churn2_test.arff'))
test = pd.DataFrame(arff_test['data'])
a = arff_test['attributes']
test.columns = [i[0] for i in a] 
test = test.replace(to_replace)
Y_test = test["LEAVE"]
X_test = test.loc[:, test.columns != "LEAVE"]

test.head()

,COLLEGE,INCOME,OVERAGE,LEFTOVER,HOUSE,HANDSET_PRICE,OVER_15MINS_CALLS_PER_MONTH,AVERAGE_CALL_DURATION,REPORTED_SATISFACTION,REPORTED_USAGE_LEVEL,CONSIDERING_CHANGE_OF_PLAN,PROFITABILITY,LEAVE
0,0,100548.0,49.0,8.0,668230.0,848.0,4.0,5.0,2,2,-2,2.61,1
1,1,31872.0,58.0,0.0,280278.0,150.0,3.0,10.0,-1,-1,-2,3.42,0
2,1,59197.0,58.0,23.0,590473.0,300.0,5.0,5.0,-2,1,-2,2.27,0
3,1,155244.0,0.0,22.0,432265.0,498.0,1.0,4.0,-2,1,2,5.66,0
4,0,132197.0,204.0,39.0,360448.0,794.0,21.0,1.0,-2,-2,2,3.13,1


## Deliverable #1: Accuracy and Cross-validation
Report the accuracy of the classifier on:
- the training data
- the test set churn2_test.arff
- 10-fold cross-validation
- 66% train 34% split of the training data

In [25]:
# accuracy on the training data:
acc_train = my_tree.score(X_train, Y_train)

# accuracy on the test data:
acc_test = 

# accuracy on 10-fold CV:
# hint: use model_selection.cross_val_score() with cv = 10 
acc_10cv = 

# accuracy on 66% split:
acc_66pct = 

print("accuracy on the training data : " + str(acc_train))
print("accuracy on the test data     : " + str(acc_test))
print("accuracy on the 10-fold CV    : " + str(acc_10cv))
print("accuracy on the 66% split     : " + str(acc_66pct))


accuracy on the training data : 1.0
accuracy on the test data     : 0.99
accuracy on the 10-fold CV    : 0.98
accuracy on the 66% split     : 0.99


## Deliverable #2.1: Construct a Fitting Curve
One way to control the complexity is by setting the depth as shown above. Another way, as shown in class is by choosing different values for the minimum number of training instances in each leaf of the tree. In sklean's DecisionTreeClassifier this parameter is called "min_samples_leaf".

- Plot a fitting curve for the train set, test set, and 3-fold cross validation with a range of values for max_depth to find the optimal complexity. 
- Next, repeat the above with a range of values for **min_samples_leaf** to find the optimal complexity. 



In [26]:
# Show work here

## Deliverable #2.2: Evaluation
Based on the above graphs:
- What value would you use for max depth?
- What value would you use for min leaf size?
- Which complexity parameter would you choose (max depth or min leaf size)?


### Answer:

Your own work!

## Deliverable #2.3: Learning Curves
Learning curves determine how much data you realistically need to train your model.
* Randomize the rows
* Use train_test_split () to select a range of percentages of the training data to use to fit the model
* Plot training size vs. accuracy on the test, train, and 3-fold cv

The X-axis of your chart should be "number for instances used"

In [ ]:
# Show work here

## Deliverable #3.1: Evaluation
Would you recommend your firm spend money to collect data on more customers?


### Answer:

Your own work!

## Deliverable #3.2: Interpreting Trees
Build and show a tree that is small enough to plot.
(Use the package graphviz to plot the tree)

In [ ]:
# Show work here

Which attribute is most informative?

In [ ]:
# Show work here

Explain in 2-3 sentences why that feature was chosen.